<h1>Problem 1:</h1>
Write a function that reads timeseries pricing data from a file into a pandas dataframe and then groups the data as follows:
<li>The arguments to the function are the filename and a threshold number
<li>The function reads the data in the file and creates a new column "pct_change" with the one day percent change
<li>Then groups the data into four categories:
<ul>
<li>"High+" if the percent change is greater than the threshold 
<li>"Low+" if the percent change is zero or positive and less than or equal to the threshold% 
<li>"Low-" if the percent change is negative but greater than or equal to -1 * the threshold
<li>"High-" if the percent change is less than -1 * the threshold
</ul>
<li>The function should return a dataframe that contains three columns (count, mean, stdev) and four index values (High+, High-, Low+, Low-)
    <p><b>Note: </b>we have to deal with nan percent changes. For the purposes of this assignment, you can either classify it as a High- (in which case you'll get the results below) or classify the one Nan in the pct change column as "unknown" (your results for the High- row below will be slightly different).<p>
For the sample data your function should return a dataframe with the following values:

<pre>
        count	mean	stdev
High+	49	75.603826	13.270096
High-	42	75.658000	15.283640
Low+	101	74.644790	11.997251
Low-	58	72.191838	9.949705
</pre>



<h3>read_csv</h3>
The pandas <span style="color:red">read_csv</span> function reads data from a delimited file (default delimiter = comma) into a pandas dataframe. It automatically detects a header line (use header=None if the file does not contain a header)
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">read_csv documentation</a>


<h2>Sample data</h2>
Use the attached SBUX.csv file

In [29]:
def change_groups(datafile,threshold):
    import pandas as pd
    import numpy as np
    # read in data as dataframe without column names
    df = pd.read_csv(datafile, header=None)
    # set column names
    df.columns = ['Date', 'Price']
    
    # create column for 1 day percent change
    df['1 Day Percent Change'] = df['Price'].pct_change()
    
    # multiply percent change by 100 to be in the same form as the threshold
    df['1 Day Percent Change'] = df['1 Day Percent Change'] * 100
    
    # inner grouping definition used to designate every percent change into its respective group according to the defintions
    # of the four categories outlined above
    def GroupColFunc(df, ind, col, threshold):
        if df[col].loc[ind] > threshold:
            return 'High+'
        elif df[col].loc[ind] == 0 or (df[col].loc[ind] > 0 and df[col].loc[ind] <= threshold):
            return 'Low+'
        elif df[col].loc[ind] < 0 and df[col].loc[ind] >= -1 * threshold:
            return 'Low-'
        else:
            return 'High-'
        
    # groups by each category in the inner definition by using a lambda function
    grouped = df.groupby(lambda x: GroupColFunc(df, x, '1 Day Percent Change', threshold))
    
    # creates each group
    count = grouped.count()['Date']
    mean = grouped.mean()['Price']
    stdev = grouped.std()['Price']
    
    
    return pd.DataFrame({'count':count,'mean':mean,'stdev':stdev})


In [30]:
change_groups("SBUX.csv",1.0)

,count,mean,stdev
High+,49,75.603826,13.270096
High-,42,75.658000,15.283640
Low+,101,74.644790,11.997251
Low-,58,72.191838,9.949705


<h1>Problem 2</h1>
A manufacturer has data on orders from customers and product prices in two dataframes (see below). They want to use this data to answer the following questions:
<ol>
    <li>Which customer is responsible for the most revenue</li>
    <li>Which customer is responsible for the highest profit</li>
    <li>Which product is responsible for the highest (dollar) profit</li>
    <li>Which customer and product combination is responsible for the most orders</li>
</ol>
<p>
Obviously, your code should work for any actual data values!
<p>For the data below: your answers should be:
    
<pre>
Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
</pre>

<h2>Useful functions:</h2>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">pd.sort_values</a> Use the <span style="color:red">by</span> keyword to chose a sort column and <span style="color:red">ascending=False</span> to sort in descending order</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">pandas dataframe join</a> the last example on the linked page is probably what you need here!</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a></li>

In [31]:
import pandas as pd
import numpy as np
order_data = pd.DataFrame([["001","010",24],
                           ["007","012",35],
                           ["001","011",15],
                           ["005","010",30],
                           ["007","011",17],
                           ["005","011",81],
                           ["001","010",32],
                           ["007","012",89],
                           ["001","010",16],
                           ["001","010",33]],columns=["customer","product","amount"])
products = pd.DataFrame([['010',22.3,17.2],
                        ['011',11.7,5.5],
                        ['012',62.5,61.4]],columns=['product','price','cost'])                     

In [32]:
# joins the data on the column, product
df_joined = order_data.join(products.set_index('product'), on='product')

# defines and creates a column for revenue
df_joined['revenue'] = df_joined['amount'] * df_joined['price']

# defines and creates a column for profit
df_joined['profit'] = df_joined['amount'] * df_joined['price'] - df_joined['amount'] * df_joined['cost']

# groups by customer and extracts the revenue
customer_revenue = df_joined.groupby('customer')['revenue']

# groups by customer and extracts the profit
customer_profit = df_joined.groupby('customer')['profit']

# groups by product and extracts the profit
product_profit = df_joined.groupby('product')['profit']

# groups by customer and product
customer_orders = df_joined.groupby(['customer', 'product'])

# prints the customer with maximum revenue
print("Customer with most revenue: " + customer_revenue.sum().idxmax())

# print the customer with the maximum profit
print("Customer with most profit: " + customer_profit.sum().idxmax())

# prints the product with the maximum profit
print("Product with most profit: " + product_profit.sum().idxmax())

# prints the name of the customer with the most number of order, the product used, and the number of orders
print("Customer " + customer_orders['product'].count().idxmax()[0] + " with product " + 
      customer_orders['product'].count().idxmax()[1] + " with " + str(customer_orders['product'].count().max()) +
     " is the most ordered customer product pair")

Customer with most revenue: 007
Customer with most profit: 005
Product with most profit: 011
Customer 001 with product 010 with 4 is the most ordered customer product pair
